## Window Theorem (3)

For two separation-identical aircraft $i$ and $j$ where $r_i \le r_j$ and $lt_i \le lt_j$, a *complete-order* can be inferred such that $i$ should precede $j$. Since an aircraft can always be delayed to meet the start of its hard time window, and $t_x \le t'_x$ for all aircraft in corresponding positions of sequences $s$ and $s'$, any time-window violation is no worse in $s$ than in $s'$. Hence, sequencing $i$ before $j$ cannot increase the number or severity of hard time-window violations. Firstly, we import (relevant) libraries, and initialise a `Solver` instance.


In [1]:
from z3 import *
from itertools import product
from functools import reduce

s = Solver()


We define the symbolic entities involved: five aircraft identifiers (p1, i, p2, j, p3) and mappings for release times $R$, mappings for end of hard time windows $L T$ and the asymmetric separation matrix $\delta$, i.e.
- $\delta \in \mathbb{Z} \to \mathbb{Z} \to \mathbb{R}$ (separation-matrix)
- $r_i \in \mathbb{Z} \to \mathbb{R}$ (release time)
- $b_i \in \mathbb{Z} \to \mathbb{R}$ (base time)



In [2]:

# Introduce aircraft sequence {p1, i, p2, j, p3}
p1, i, p2, j, p3 = Ints('p1 i p2 j p3')
aircraft = [p1, i, p2, j, p3]

# Define some mappings for R (release time) LT (end of time window) and δ (minimum separation)
R  = {ac: Real(f"R_{ac}")  for ac in aircraft}
LT = {ac: Real(f"LT_{ac}") for ac in aircraft}
δ   = {(x, y): Real(f"δ_{x}_{y}") for x, y in product(aircraft, aircraft)}


Then, we constrain some properties of $R$, $LT$ and $\delta$, as follows:

1. Non-negativity of release times / $r_x$:
$$ r_x \geq 0 \quad \forall x \in S $$

2. Non-negativity of end of time windows / $lt_i$
$$ b_x \geq 0 \quad \forall x \in S $$

3. Non-negativity of $\delta$
$$ \delta(x, y) \geq 0 \quad \forall x \in (S \times S) $$

4. Identical separations between `i` and `i` $\delta_i = \delta_j$
$$ \delta(i, x) = \delta(j, x) \wedge \delta(x, i) = \delta(x, j) \quad \forall x \in S $$

In [3]:

# Each aircraft ID should be unique ∀(x, y) ∈ S × S, x ≠ y
s.add(Distinct(*aircraft))



# R & LT are non-negative real numbers.
for x in aircraft:
    s.add(R[x] > 0, LT[x] > 0)



# δ maps into a non-negative codomain.
for (x, y) in product(aircraft, aircraft):
    s.add(δ[x, y] >= 0)


# δ-identical i/j
for x in aircraft:
    s.add(δ[i, x] == δ[j, x])
    s.add(δ[x, i] == δ[x, j])


We define a recursive symbolic function `compute_T` which encodes the earliest feasible start time of each aircraft in a sequence. This is an encoding of the $t_i$ entity from the paper, to implement this we define some intermediary $m a x$ functions.

In [4]:

# helper (symbolic max)
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs): return reduce(lambda a, b: zmax(a, b), xs)

def compute_T(seq):
    T = {seq[0]: R[seq[0]]}
    for k in range(1, len(seq)):
        preds = seq[:k]
        T[seq[k]] = zmax(R[seq[k]],
                         zmax_list([T[p] + δ[p, seq[k]] for p in preds]))
    return T


Define a function `window_violations` to calculate a symbolic representation of the number of window violations in that sequence/t-assignment.

In [5]:

def window_violations(T):
    return Sum([If(T[a] > LT[a], 1, 0) for a in aircraft])


Now, encode the theorem preconditions, that is:

$$s = \{ p1, i, p2, j, p3 \}$$
$$s' = \{ p1, j, p2, i, p3 \}$$
$$r_i < r_j$$
$$lt_i < lt_j$$

We then introduce some new symbols:

- $T, T'$ represent the take off times of aircraft in $s$ and $s'$ repsectively.
- $WV, WV'$ represent the total number of time window violations (where $T(x) > LT(x)$ for any aircraft $x$ in the sequence).

We also encode that $s'$ is feasible, this to show that is $s'$ is feasible, then $s$ will always be better, and will always be feasible (if $s'$ is).
$$WV' == 0$$

Now, we want the solver to prove the non-existence of a case where $WV' == 0$ and $WV > WV'$, so we encode the 'hypothesis' constraint - that is, find a case where $s$ is infeasible.

$$WV > 0$$


In [6]:

# encode s and s'
S       = [p1, i, p2, j, p3]
S_prime = [p1, j, p2, i, p3]

T       = compute_T(S)
T_prime = compute_T(S_prime)

WV      = window_violations(T)
WV_p    = window_violations(T_prime)



# theorem assumptions
s.add(R[i] < R[j])
s.add(LT[i] < LT[j])



# S' Is feasible (no window violations in S')
s.add(WV_p == 0)



# hypothesis
s.add(WV > WV_p)


Finally check the solver.

In [7]:

if s.check() == z3.sat:
    model = s.model()
    print("\nTheorem holds SAT (counterexample found).\nModel:")
    for d in model.decls():
        print(f" - {d.name()} = {model[d]}")
elif s.check() == z3.unsat:
    print("\nUNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance if s' is feasible.")



UNSAT — Theorem 3 holds: a before b cannot worsen hard-window compliance if s' is feasible.
